------------------------------
1 推荐模型的分类
------------------------------

推荐系统的研究已经相当广泛,也存在很多设计方法。最为流行的两种方法是基于内容的过滤和协同过滤。另外,排名模型等近期也受到不少关注。实践中的方案很多是综合性的,它们将多种方法的元素合并到一个模型中或是进行组合。

--------------------
1.1 基于内容的过滤
----------------------

基于内容的过滤利用物品的内容或是属性信息以及某些相似度定义,来求出与该物品类似的物品。这些属性值通常是文本内容(比如标题、名称、标签及该物品的其他元数据)。对多媒体来说,可能还涉及从音频或视频中提取的其他属性。

类似地,对用户的推荐可以根据用户的属性或是描述得出,之后再通过相同的相似度定义来与物品属性做匹配。比如,用户可以表示为他所接触过的各物品属性的综合。该表示可作为该用户的一种描述。之后可以用它来与物品的属性进行比较以找出符合用户描述的物品。


-----------------
1.2协同过滤
-----------------

协同过滤是一种借助众包智慧的途径。它利用大量已有的用户偏好来估计用户对其未接触过的物品的喜好程度。其内在思想是相似度的定义。

在基于用户的方法的中,如果两个用户表现出相似的偏好(即对相同物品的偏好大体相同),那就认为他们的兴趣类似。要对他们中的一个用户推荐一个未知物品,便可选取若干与其类似的用户并根据他们的喜好计算出对各个物品的综合得分,再以得分来推荐物品。其整体的逻辑是,如果其他用户也偏好某些物品,那这些物品很可能值得推荐。

同样也可以借助基于物品的方法来做推荐。这种方法通常根据现有用户对物品的偏好或是评级情况,来计算物品之间的某种相似度。这时,相似用户评级相同的那些物品会被认为更相近。一旦有了物品之间的相似度,便可用用户接触过的物品来表示这个用户,然后找出和这些已知物品相似的那些物品,并将这些物品推荐给用户。同样,与已有物品相似的物品被用来生成一个综合得分,而该得分用于评估未知物品的相似度。



基于用户或物品的方法的得分取决于若干用户或是物品之间依据相似度所构成的集合(即邻居),故它们也常被称为最近邻模型。

最后,也存在不少基于模型的方法是对“用户物品”偏好建模。这样,对未知“用户物品”组合上应用该模型便可得出新的偏好。


----------------
1.3 矩阵分解
----------------

Spark推荐模型库当前只包含基于矩阵分解(matrix factorization)的实现,由此我们也将重点关注这类模型。它们有吸引人的地方。首先,这些模型在协同过滤中的表现十分出色。而在Netflix Prize等知名比赛中的表现也很拔尖。

-----------------
2 提取有效特征
-----------------

这里,我们将采用显式评级数据,而不使用其他用户或物品的元数据以及“用户物品”交互数据。这样,所需的输入数据就只需包括每个评级对应的用户ID、影片ID和具体的星级。本文中使用显式数据也就是用户对movie的rating信息，这个数据来源于网络上的MovieLens标准数据集

In [1]:
from pyspark import SparkConf,SparkContext
from pyspark import rdd
conf=SparkConf().setMaster("local[*]").setAppName("First_APP")
sc=SparkContext(conf=conf)

In [2]:
rawData = sc.textFile("E:/machine_data/spark_test_data/ml-100k/u.data")
print (rawData.first())
rawRatings = rawData.map(lambda x: x.split('\t'))
rawRatings.take(5)
#数据分别是userId，itemId，rating和timestamp

196	242	3	881250949


[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [3]:
#导入Rating、ALS模块(最小二乘)，用于后面建模
from pyspark.mllib.recommendation import Rating, ALS
ratings = rawRatings.map(lambda x: Rating(int(x[0]),int(x[1]),float(x[2])))
ratings.take(5)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0),
 Rating(user=22, product=377, rating=1.0),
 Rating(user=244, product=51, rating=2.0),
 Rating(user=166, product=346, rating=1.0)]

------------------------
3 训练推荐模型
========================



现在可以开始训练模型了,所需的其他参数有以下几个。




 rank:对应ALS模型中的因子个数,也就是在低阶近似矩阵中的隐含特征个数。因子个数一般越多越好。但它也会直接影响模型训练和保存时所需的内存开销,尤其是在用户和物品很多的时候。因此实践中该参数常作为训练效果与系统开销之间的调节参数。通常,其合理取值为10到200。
   对应ALS模型中的因子个数，即“两个小矩阵和”中的k

 iterations:对应运行时的迭代次数。ALS能确保每次迭代都能降低评级矩阵的重建误差,但一般经少数次迭代后ALS模型便已能收敛为一个比较合理的好模型。这样,大部分情况下都没必要迭代太多次(10次左右一般就挺好)。

 lambda:该参数控制模型的正则化过程,从而控制模型的过拟合情况。其值越高,正则化越严厉。该参数的赋值与实际数据的大小、特征和稀疏程度有关。和其他的机器学习模型一样,正则参数应该通过用非样本的测试数据进行交叉验证来调整。
   控制模型的正则化过程，从而控制模型的过拟合情况。




作为示例,这里将使用的rank、iterations和lambda参数的值分别为50、10和0.01:


In [4]:
model = ALS.train(ratings, 50, 10, 0.01)
userFeatures = model.userFeatures()
userFeatures.take(2)

[(4,
  array('d', [0.6825532913208008, -0.08277151733636856, -0.5213354229927063, -0.11160024255514145, 0.23231713473796844, 0.35927271842956543, -0.15395067632198334, 0.3292972445487976, -0.10530982911586761, -0.45273345708847046, 0.13377642631530762, 0.4834160804748535, 0.12130600959062576, 0.1699017584323883, -0.19415707886219025, 0.4661986231803894, -0.2508096694946289, -0.22582422196865082, 0.008530866354703903, -0.4816049337387085, 0.10357516258955002, -0.528141438961029, -0.14657121896743774, -0.022775448858737946, -0.04532771185040474, 0.3767406642436981, 0.6552073359489441, -0.0724124163389206, 0.254604309797287, -0.311857134103775, -0.18600477278232574, 0.30310359597206116, -0.5999680161476135, 0.4510653018951416, 0.10150831192731857, 0.043596796691417694, 0.10472331941127777, -0.17609846591949463, 0.25088027119636536, 0.0338578037917614, 0.21619196236133575, -0.013780420646071434, -0.005915130954235792, -0.18161839246749878, -0.01525501161813736, 0.3262485861778259, -0.19702

In [5]:
productFeatures = model.productFeatures()
productFeatures.take(2)

[(4,
  array('d', [0.7801207304000854, 0.8949319124221802, -0.9001529216766357, -0.03314482048153877, -0.10774897038936615, 0.26987990736961365, 0.8206261992454529, 0.6646865606307983, 0.6464346051216125, 0.4607182443141937, 0.4361865520477295, -0.47185373306274414, 0.5803003907203674, 1.125365972518921, -0.14290845394134521, 0.5238562226295471, 0.2841642200946808, -1.4662121534347534, -0.04852978512644768, -0.9764675498008728, -1.0508042573928833, -1.356313705444336, -0.9136911630630493, -0.19573234021663666, -0.467304527759552, 0.24956005811691284, 0.3283959627151489, -0.16028010845184326, 0.6407354474067688, -0.16508187353610992, -0.017425725236535072, -0.28238147497177124, -0.6315313577651978, -0.4000170826911926, -0.3455822467803955, -0.19245243072509766, 1.189756989479065, 0.4005597233772278, -1.3466495275497437, 1.1558536291122437, 0.21020039916038513, -0.2203013151884079, 0.2319221943616867, -0.42747271060943604, 0.9779833555221558, -0.04015105590224266, -0.77442866563797, 0.11

In [6]:
print (model.userFeatures().count())
print (model.productFeatures().count())

943
1682




使用隐式反馈数据训练模型




MLlib中标准的矩阵分解模型用于显式评级数据的处理。若要处理隐式数据,则可使用trainImplicit函数。其调用方式和标准的train模式类似,但多了一个可设置的alpha参数（也是一个正则化参数,lambda应通过测试和交叉验证法来设置）。
alpha参数指定了信心权重所应达到的基准线。该值越高则所训练出的模型越认为用户与他所没评级过的电影之间没有相关性。 

-----------------
4 使用推荐模型
-----------------

--------------
4.1用户推荐
--------------

有了训练好的模型后便可用它来做预测。预测通常有两种:为某个用户推荐物品,或找出与某个物品相关或相似的其他物品。






从MovieLens 100k数据集生成电影推荐

MLlib的推荐模型基于矩阵分解,因此可用模型所求得的因子矩阵来计算用户对物品的预计评级。下面只针对利用MovieLens中显式数据做推荐的情形,使用隐式模型时的方法与之相同。


In [7]:
print (len(userFeatures.first()[1]))
predictRating = model.predict(789,123)
print (predictRating)

50
4.643654084950845


可以看到,该模型预测用户789对电影123的评级为3.658。




predict函数同样可以以(user, item)ID对类型的RDD对象为输入,这时它将为每一对都生成相应的预测得分。我们可以借助这个函数来同时为多个用户和物品

进行预测。




要为某个用户生成前K个推荐物品,可借助MatrixFactorizationModel所提供的recommendProducts函数来实现。该函数需两个输入参数:user和num。其中user

是用户ID,而num是要推荐的物品个数。




返回值为预测得分最高的前num个物品。这些物品的序列按得分排序。该得分为相应的用户因子向量和各个物品因子向量的点积。




现在,计算给用户789推荐的前10个物品，代码如下：






In [8]:
topKRecs = model.recommendProducts(789,10)
print ('给用户userId推荐其喜欢的item：')
for rec in topKRecs:
    print (rec)

给用户userId推荐其喜欢的item：
Rating(user=789, product=187, rating=5.614482532373878)
Rating(user=789, product=517, rating=5.590638326236133)
Rating(user=789, product=654, rating=5.406477736839683)
Rating(user=789, product=302, rating=5.307652387214368)
Rating(user=789, product=518, rating=5.272251750771314)
Rating(user=789, product=195, rating=5.221785802007747)
Rating(user=789, product=48, rating=5.215569331702351)
Rating(user=789, product=429, rating=5.168334110934902)
Rating(user=789, product=127, rating=5.054994031204602)
Rating(user=789, product=129, rating=5.0350278294993585)


检验推荐内容




要直观地检验推荐的效果,可以简单比对下用户所评级过的电影和被推荐的那些电影

下面查看用户789评级过的电影，代码如下：












In [9]:
moviesForUser = ratings.groupBy(lambda x : x.user).mapValues(list)
#print ratings.groupBy(lambda x : x.user).mapValues(list).lookup(789)
#print ratings.groupBy(lambda x : x.user).map(lambda (x,y): (x, list(y))).lookup(789)
#print moviesForUser
moviesForUser=moviesForUser.collect()
moviesForUser=sc.parallelize([(x, y) for (x, y) in moviesForUser]).lookup(789)

print ('用户对%d部电影进行了评级'%len(moviesForUser[0]))
print ('源数据中用户(userId=789)喜欢的电影(item)：')
#for i in sorted(moviesForUser[0],key=lambda x:x.rating,reverse=True):
for i in moviesForUser[0]:
    print (i.product,i.rating)



用户对33部电影进行了评级
源数据中用户(userId=789)喜欢的电影(item)：
1012 4.0
127 5.0
475 5.0
93 4.0
1161 3.0
286 1.0
293 4.0
9 5.0
50 5.0
294 3.0
181 4.0
1 3.0
1008 4.0
508 4.0
284 3.0
1017 3.0
137 2.0
111 3.0
742 3.0
248 3.0
249 3.0
1007 4.0
591 3.0
150 5.0
276 5.0
151 2.0
129 5.0
100 5.0
741 5.0
288 3.0
762 3.0
628 3.0
124 4.0


In [13]:
print (moviesForUser[0])
print (len(moviesForUser[0]))

[Rating(user=789, product=1012, rating=4.0), Rating(user=789, product=127, rating=5.0), Rating(user=789, product=475, rating=5.0), Rating(user=789, product=93, rating=4.0), Rating(user=789, product=1161, rating=3.0), Rating(user=789, product=286, rating=1.0), Rating(user=789, product=293, rating=4.0), Rating(user=789, product=9, rating=5.0), Rating(user=789, product=50, rating=5.0), Rating(user=789, product=294, rating=3.0), Rating(user=789, product=181, rating=4.0), Rating(user=789, product=1, rating=3.0), Rating(user=789, product=1008, rating=4.0), Rating(user=789, product=508, rating=4.0), Rating(user=789, product=284, rating=3.0), Rating(user=789, product=1017, rating=3.0), Rating(user=789, product=137, rating=2.0), Rating(user=789, product=111, rating=3.0), Rating(user=789, product=742, rating=3.0), Rating(user=789, product=248, rating=3.0), Rating(user=789, product=249, rating=3.0), Rating(user=789, product=1007, rating=4.0), Rating(user=789, product=591, rating=3.0), Rating(user

In [14]:
l = range(1000)
rdd = sc.parallelize(zip(l, l), 10)
print (type(rdd))
print (rdd.take(4))
rdd.lookup(42)  #slow

<class 'pyspark.rdd.RDD'>
[(0, 0), (1, 1), (2, 2), (3, 3)]


[42]

In [32]:
sorted = rdd.sortByKey()
sorted.lookup(42) #fast

[42]

In [98]:
rdd2 = sc.parallelize([('a',1),('a', 2),('b',1),('b', 2)])
rdd2.lookup('a')

[1, 2]

In [80]:
rdd2 = sc.parallelize([('a',1),('a', 2),('b',1),('b', 2)]).groupByKey()
rdd2.id()

395

In [15]:
movies = sc.textFile("E:/machine_data/spark_test_data/ml-100k/u.item")
titles = movies.map(lambda line: (int(line.split('|')[0]),line.split('|')[1])).collectAsMap()
print (titles[1])

Toy Story (1995)


In [16]:
movies.first()

'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [86]:
print (titles)

{1: 'Toy Story (1995)', 2: 'GoldenEye (1995)', 3: 'Four Rooms (1995)', 4: 'Get Shorty (1995)', 5: 'Copycat (1995)', 6: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 7: 'Twelve Monkeys (1995)', 8: 'Babe (1995)', 9: 'Dead Man Walking (1995)', 10: 'Richard III (1995)', 11: 'Seven (Se7en) (1995)', 12: 'Usual Suspects, The (1995)', 13: 'Mighty Aphrodite (1995)', 14: 'Postino, Il (1994)', 15: "Mr. Holland's Opus (1995)", 16: 'French Twist (Gazon maudit) (1995)', 17: 'From Dusk Till Dawn (1996)', 18: 'White Balloon, The (1995)', 19: "Antonia's Line (1995)", 20: 'Angels and Insects (1995)', 21: 'Muppet Treasure Island (1996)', 22: 'Braveheart (1995)', 23: 'Taxi Driver (1976)', 24: 'Rumble in the Bronx (1995)', 25: 'Birdcage, The (1996)', 26: 'Brothers McMullen, The (1995)', 27: 'Bad Boys (1995)', 28: 'Apollo 13 (1995)', 29: 'Batman Forever (1995)', 30: 'Belle de jour (1967)', 31: 'Crimson Tide (1995)', 32: 'Crumb (1994)', 33: 'Desperado (1995)', 34: 'Doom Generation, The (1995)', 35:

In [17]:
for i,rec in enumerate(topKRecs):
    print ('rank:'+str(i)+' '+str(titles[rec.product])+':'+str(rec.rating))




rank:0 Godfather: Part II, The (1974):5.614482532373878
rank:1 Manhattan (1979):5.590638326236133
rank:2 Chinatown (1974):5.406477736839683
rank:3 L.A. Confidential (1997):5.307652387214368
rank:4 Miller's Crossing (1990):5.272251750771314
rank:5 Terminator, The (1984):5.221785802007747
rank:6 Hoop Dreams (1994):5.215569331702351
rank:7 Day the Earth Stood Still, The (1951):5.168334110934902
rank:8 Godfather, The (1972):5.054994031204602
rank:9 Bound (1996):5.0350278294993585


使用recommendProducts来为用户789推荐top10的items，其items顺序为降序。MoviesForUser是从ratings数据中找出的用户789rating最高的数据，仔细看下发现数据和我们的ratings里面找出的数据貌似一个都没有相同的，那么是不是说明我们的算法不给力呢？这个可不一定，想想看，如果推荐系统只是推荐给你看过的电影，那么它一定是一个失败的，并且完全对系统的kpi数据无提升作用，前面提到，MF的实质是通过latent feature去找到与用户过去偏好高的有某些隐性相同特征的电影（这些由整体用户的集体智慧得到），比如可能是某一类型的电影、又或者相同的演员等等，所以这里不能说明推荐系统不给力，但是确实也很难具有解释性。 

-----------------------
4.2 物品推荐
-----------------------

物品推荐是为回答如下问题:给定一个物品,有哪些物品与它最相似?这里,相似的确切定义取决于所使用的模型。大多数情况下,相似度是通过某种方式比较表示两个物品的向量而得到的。常见的相似度衡量方法包括皮尔森相关系数(Pearson correlation)、针对实数向量的余弦相似度(cosine similarity)和针对二元向量的杰卡德相似系数(Jaccard similarity)。



从MovieLens 100k数据集生成相似电影




MatrixFactorizationModel当前的API不能直接支持物品之间相似度的计算。所以我们要自己实现。这里我们使用余弦相似度来衡量相似度。


In [18]:
import numpy as np
def cosineSImilarity(x,y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
testx = np.array([1.0,2.0,3.0])
print (cosineSImilarity(testx,testx))

1.0


In [19]:
#下面以itemId = 567为例，计算所有与物品itemId = 567的的相似度，并取其相似度最高的前十个，代码如下:
itemId = 567
itemFactor = model.productFeatures().lookup(itemId)[0]
print (itemFactor)

#print model.productFeatures().collect()
#print model.productFeatures().take(1)
sims = model.productFeatures().map(lambda kv:(kv[0],cosineSImilarity(np.array(kv[1]),
             np.array(itemFactor))))

sims.sortBy(lambda kv:kv[1],ascending=False).take(10)


array('d', [1.3959728479385376, -0.32170650362968445, -1.589107871055603, -0.24735496938228607, -0.09870206564664841, 1.2067121267318726, 0.8226302266120911, -0.011584000661969185, 0.7589138150215149, -0.16595466434955597, 0.11890197545289993, -0.6180840134620667, 0.3590054214000702, 0.4999312162399292, 0.2935958504676819, 0.012971253134310246, 0.31235596537590027, -0.9185784459114075, -0.3205436170101166, -0.26075801253318787, -0.7303558588027954, -0.7694108486175537, -0.5730422735214233, 0.07830443233251572, -0.16548651456832886, 0.15430086851119995, -0.6806657314300537, 0.5374705195426941, 0.49215051531791687, 0.11401540786027908, 0.631446897983551, 0.1660444140434265, -0.03138217329978943, -0.35071438550949097, 0.11190352588891983, -0.2576572895050049, 0.7578253149986267, -0.6897003650665283, -0.5767093300819397, -0.5592586398124695, 0.477077454328537, -0.13024748861789703, 0.8320048451423645, -0.3771703839302063, -0.1656838208436966, 0.025560475885868073, -0.4893408715724945, 0.03

[(567, 1.0),
 (642, 0.70063739485500198),
 (257, 0.69057993350143687),
 (636, 0.68806967734139302),
 (670, 0.68437271936248811),
 (1169, 0.67968316901992409),
 (106, 0.66842863474932324),
 (355, 0.66669706468211187),
 (7, 0.66549710867942158),
 (288, 0.66441772300960988)]

-----------------------
5 推荐模型效果的评估
-----------------------



如何知道训练出来的模型是一个好模型?这就需要某种方式来评估它的预测效果。评估指标(evaluation metric)指那些衡量模型预测能力或准确度的方法。它们有些直接度量模型的预测目标变量的好坏(比如均方差),有些则关注模型对那些其并未针对性优化过但又十分接近真实应用场景数据的预测能力(比如平均准确率)。



评估指标提供了同一模型在不同参数下,又或是不同模型之间进行比较的标准方法。通过这些指标,人们可以从待选的模型中找出表现最好的那个模型。本博文主要介绍推荐系统和协同过滤模型里常用的两个指标:均方差以及K值平均准确率。


--------------
5.1 均方差
--------------



均方差(Mean Squared Error,MSE)直接衡量“用户物品”评级矩阵的重建误差。它也是一些模型里所采用的最小化目标函数,特别是许多矩阵分解类方法,比如ALS。因此,它常用于显式评级的情形。




它的定义为各平方误差的和与总数目的商。其中平方误差是指预测到的评级与真实评级的差值的平方。下面以用户789为例做讲解。现在从之前计算的moviesForUser这个Ratings集合里找出该用户的第一个评级:


In [20]:
actual = moviesForUser[0][0]
actualRating = actual.rating
print ('用户789对电影1012的实际评级',actualRating)
predictedRating = model.predict(789, actual.product)
print ('用户789对电影1012的预测评级',predictedRating)
squaredError = np.power(actualRating-predictedRating,2)
print ('实际评级与预测评级的MSE',squaredError)

用户789对电影1012的实际评级 4.0
用户789对电影1012的预测评级 3.978939980004587
实际评级与预测评级的MSE 0.000443524442207


要计算整个数据集上的MSE,需要对每一条(user, movie, actual rating, predictedrating)记录都计算该平均误差,然后求和,再除以总的评级次数。具体实现如下:

In [21]:
userProducts = ratings.map(lambda rating:(rating.user,rating.product))
print ('实际的评分:',userProducts.take(5))

#预测所有用户对电影的相应评分
print (model.predictAll(userProducts).collect()[0])
predictions = model.predictAll(userProducts).map(lambda rating:((rating.user,rating.product), rating.rating))
print ('预测的评分:',predictions.take(5))

ratingsAndPredictions = ratings.map(lambda rating:((rating.user,rating.product),rating.rating)).join(predictions)
print ('组合预测的评分和实际的评分:',ratingsAndPredictions.take(5))

MSE = ratingsAndPredictions.map(lambda kv:np.power(kv[1][0]-kv[1][1],2)).reduce(lambda x,y:x+y)/ratingsAndPredictions.count() 
print ('模型的均方误差:',MSE )
print ('模型的均方根误差:',np.sqrt(MSE))

实际的评分: [(196, 242), (186, 302), (22, 377), (244, 51), (166, 346)]
Rating(user=316, product=1084, rating=4.087165993469001)
预测的评分: [((316, 1084), 4.087165993469001), ((504, 1084), 3.9465249959951514), ((424, 1084), 4.88215498786934), ((541, 1084), 3.9893203891299764), ((181, 1084), 1.9961869306829314)]
组合预测的评分和实际的评分: [((711, 707), (5.0, 5.008434839376742)), ((650, 622), (3.0, 3.0118055017330714)), ((731, 1), (2.0, 2.4430638270113585)), ((752, 316), (3.0, 3.0000103178325035)), ((18, 428), (3.0, 3.3634323463366558))]
模型的均方误差: 0.0843347796363
模型的均方根误差: 0.290404510358


In [22]:
#简单地测试一下map(lambda)多参数的情况
a=sc.parallelize([((1,2),(2,3)),((2,3),(3,4))])
b=a.map(lambda kv:(kv[0][0]+kv[1][0],kv[0][1]+kv[1][1]))

In [23]:
b.collect()

[(3, 5), (5, 7)]

-------------------
5.2 K值平均准确率
-------------------



K值平均准确率(MAP)的意思是整个数据集上的K值平均准确率(Average Precision at Kmetric,MAP)的均值。APK是信息检索中常用的一个指标。它用于衡量针对某个查询所返回的“前K个”文档的平均相关性。对于每次查询,我们会将结果中的前K个与实际相关的文档进行比较。

用MAP指标计算时,结果中文档的排名十分重要。如果结果中文档的实际相关性越高且排名也更靠前,那APK分值也就越高。由此,它也很适合评估推荐的好坏。因为推荐系统也会计算“前K个”推荐物,然后呈现给用户。如果在预测结果中得分更高(在推荐列表中排名也更靠前)的物品实际上也与用户更相关,那自然这个模型就更好。MAP和其他基于排名的指标同样也更适合评估隐式数据集上的推荐。这里用MSE相对就不那么合适。






当用MAP来做评估推荐模型时,每一个用户相当于一个查询,而每一个“前K个”推荐物组成的集合则相当于一个查到的文档结果集合。用户对电影的实际评级便对应着文档的实际相关性。这样，APK所试图衡量的是模型对用户感兴趣和会去接触的物品的预测能力，MAP的Python脚本如下：

In [24]:
def avgPrecisionK(actual, predicted, k): 
    if len(predicted) > k:
        predK = predicted[:k]
    else:
        predK = predicted
    score = 0.0
    numHits = 0.0
    for i,p in enumerate(predK):
        #print (i,p)
        if p in actual and p not in predK:
            numHits = numHits + 1
            score = score + numHits/(i+1)
    if not actual:
        return 1.0
    else:
        return score/min(len(actual),k)



In [25]:
actualMovies = [rating.product for rating in moviesForUser[0]]
predictMovies = [rating.product for rating in topKRecs]
print ('实际的电影：',actualMovies)
print ('预测的电影：',predictMovies)



实际的电影： [1012, 127, 475, 93, 1161, 286, 293, 9, 50, 294, 181, 1, 1008, 508, 284, 1017, 137, 111, 742, 248, 249, 1007, 591, 150, 276, 151, 129, 100, 741, 288, 762, 628, 124]
预测的电影： [187, 517, 654, 302, 518, 195, 48, 429, 127, 129]


In [26]:
MAP10 = avgPrecisionK(actualMovies,predictMovies,10)
print (MAP10)

0.0


这里,APK的得分为0,这表明该模型在为该用户做相关电影预测上的表现并不理想。





全局MAPK的求解要计算对每一个用户的APK得分,再求其平均。这就要为每一个用户都生成相应的推荐列表。针对大规模数据处理时,这并不容易,但我们可以通过Spark将该计算分布式进行。不过,这就会有一个限制,即每个工作节点都要有完整的物品因子矩阵。这样它们才能独立地计算某个物品向量与其他所有物品向量之间的相关性。然而当物品数量众多时,单个节点的内存可能保存不下这个矩阵。此时,这个限制也就成了问题。




下面看一看如何求解。首先,取回物品因子向量并用它来构建一个DoubleMatrix对象


In [27]:
itemFactors = model.productFeatures().map(lambda kv:kv[1]).collect()
itemMatrix = np.array(itemFactors)
print (itemMatrix)
print (itemMatrix.shape)

[[ 0.78012073  0.89493191 -0.90015292 ...,  0.11756649 -1.24536204
  -0.25535271]
 [ 1.6917069   0.94344932 -1.26232862 ..., -0.42816687 -1.66540205
   0.15662608]
 [ 1.56437778  0.62693602 -1.27876437 ...,  0.4456602  -1.7033087
   0.44060752]
 ..., 
 [ 0.12371797 -0.01746471 -0.09462035 ...,  0.06462495 -0.1179217
   0.01217086]
 [-0.00573086  0.01318853 -0.26214176 ...,  0.25771117  0.13779758
   0.12126365]
 [ 0.14654826 -0.12719707 -0.25111821 ...,  0.04560006 -0.17178555
   0.13437787]]
(1682, 50)


这说明itemMatrix的行列数分别为1682和50。这正常,因为电影数目和因子维数分别就是这么多。接下来,我们将该矩阵以一个广播变量的方式分发出去,以便每个工作节点都能访问到:

In [28]:
#便于后面进行map计算的时候使所有map都有效，这里在实际当中的意思是把数据广播到任何一个节点
imBroadcast = sc.broadcast(itemMatrix)
print (imBroadcast.value)
print (imBroadcast.value.shape)

[[ 0.78012073  0.89493191 -0.90015292 ...,  0.11756649 -1.24536204
  -0.25535271]
 [ 1.6917069   0.94344932 -1.26232862 ..., -0.42816687 -1.66540205
   0.15662608]
 [ 1.56437778  0.62693602 -1.27876437 ...,  0.4456602  -1.7033087
   0.44060752]
 ..., 
 [ 0.12371797 -0.01746471 -0.09462035 ...,  0.06462495 -0.1179217
   0.01217086]
 [-0.00573086  0.01318853 -0.26214176 ...,  0.25771117  0.13779758
   0.12126365]
 [ 0.14654826 -0.12719707 -0.25111821 ...,  0.04560006 -0.17178555
   0.13437787]]
(1682, 50)


In [29]:
userVector = model.userFeatures().map(lambda kv:(kv[0],np.array(kv[1])))
print (userVector.take(2))
print (userVector.count())
print (userVector.collect()[0][1].shape)
userVector = userVector.map(lambda kv: (kv[0],imBroadcast.value.dot((np.array(kv[1]).transpose()))))
print (userVector.collect()[0][1].shape)
print (userVector.take(2))
userVectorId = userVector.map(lambda kv: (kv[0],[(xx,i) for i,xx in enumerate(kv[1].tolist())]))
print (userVector.collect()[0][1].shape)
print (userVector.take(2))
#print userVectorId.collect()[0]
sortUserVectorId = userVectorId.map(lambda kv:(kv[0],sorted(kv[1],key=lambda x:x[0],reverse=True)))

print (type(sortUserVectorId))
sortUserVectorRecId = sortUserVectorId.map(lambda kv: (kv[0],[xx[1] for xx in kv[1]]))

#sortUserVectorRecId.take(2)
sortUserVectorRecId.count()

[(4, array([ 0.68255329, -0.08277152, -0.52133542, -0.11160024,  0.23231713,
        0.35927272, -0.15395068,  0.32929724, -0.10530983, -0.45273346,
        0.13377643,  0.48341608,  0.12130601,  0.16990176, -0.19415708,
        0.46619862, -0.25080967, -0.22582422,  0.00853087, -0.48160493,
        0.10357516, -0.52814144, -0.14657122, -0.02277545, -0.04532771,
        0.37674066,  0.65520734, -0.07241242,  0.25460431, -0.31185713,
       -0.18600477,  0.3031036 , -0.59996802,  0.4510653 ,  0.10150831,
        0.0435968 ,  0.10472332, -0.17609847,  0.25088027,  0.0338578 ,
        0.21619196, -0.01378042, -0.00591513, -0.18161839, -0.01525501,
        0.32624859, -0.19702269, -0.33111033,  0.00490539,  0.1413146 ])), (8, array([ 0.36486393, -0.41770822, -0.43548453, -0.08969285,  0.26679993,
       -0.71609515, -0.0609786 ,  0.0515596 ,  0.3386181 ,  0.36871737,
        0.36068422,  0.5104472 , -0.09788674,  0.0860413 , -0.22378843,
        0.41948238,  0.10583912, -0.35368323,  0.035

943

In [30]:
a=[(1,(2,2)),(2,(3,3)),(0,(9,9)),(3,(4,4))]
a=sc.parallelize(a)
b = a.map(lambda kv:(kv[0],sorted(kv[1],key=lambda x:x[0],reverse=True)))
#b=a.takeOrdered(4, key=lambda x:-x[1])
a1=[(1,2),(2,3),(0,9),(3,4)]
print (sorted(a1,key=lambda x:x[1]))
#b.collect()

[(1, 2), (2, 3), (3, 4), (0, 9)]


In [10]:
a1=[(1,2),(2,3),(0,9),(3,4)]
print (type (a1))
sorted(a1,key=lambda x:x[1])

<class 'list'>


[(1, 2), (2, 3), (3, 4), (0, 9)]

In [32]:
userMovies = ratings.map(lambda rating: (rating.user,rating.product)).groupBy(lambda kv:kv[0])
print (userMovies.take(3))
#print (userMovies.collect()[0:2])
userMovies = userMovies.map(lambda kv:(kv[0], [xx[1] for xx in kv[1]] ))
#print userMovies.take(1)
#allAPK=sortUserVectorRecId.join(userMovies).map(lambda (userId,(predicted, actual)):(userId,avgPrecisionK(actual,predicted,10)))
#print allAPK.map(lambda (x,y):y).reduce(lambda x,y:x+y)/sortUserVectorRecId.count()
allAPK=sortUserVectorRecId.join(userMovies).map(lambda kv:avgPrecisionK(kv[1][1],kv[1][0],2000))
# print allAPK.take(10)
print (allAPK.reduce(lambda x,y:x+y)/allAPK.count())



[(2, <pyspark.resultiterable.ResultIterable object at 0x000001BB73FF23C8>), (4, <pyspark.resultiterable.ResultIterable object at 0x000001BB73FF2F28>), (6, <pyspark.resultiterable.ResultIterable object at 0x000001BB73FF2E10>)]
0.0


---------------------------------
5.3 使用MLlib内置的评估函数
---------------------------------



RMSE和MSE







前面我们从零开始对模型进行了MSE、RMSE和MAP三方面的评估。这是一段很有用的练习。同样, MLlib下的RegressionMetrics和RankingMetrics类也提供了相应的函数以方便模型评估。


In [33]:
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.evaluation import RankingMetrics
predictedAndTrue = ratingsAndPredictions.map(lambda kv:(kv[1][0],kv[1][1]))
print (predictedAndTrue.take(5))
regressionMetrics = RegressionMetrics(predictedAndTrue)
print ("均方误差 = %f"%regressionMetrics.meanSquaredError)
print ("均方根误差 = %f"% regressionMetrics.rootMeanSquaredError)

[(5.0, 5.008434839376742), (3.0, 3.0118055017330714), (2.0, 2.4430638270113585), (3.0, 3.0000103178325035), (3.0, 3.3634323463366558)]
均方误差 = 0.084335
均方根误差 = 0.290405




MAP




与计算MSE和RMSE一样,可以使用MLlib的RankingMetrics类来计算基于排名的评估指标。类似地,需要向我们之前的平均准确率函数传入一个键值对类型的RDD。其键为给定用户预测的推荐物品的ID数组,而值则是实际的物品ID数组。

RankingMetrics中的K值平均准确率函数的实现与我们的有所不同,因而结果会不同。但全局平均准确率(Mean Average Precision,MAP,并不设定阈值K)会和当K值较大(比如设为总的物品数目)时我们模型的计算结果相同。


In [34]:
sortedLabels = sortUserVectorRecId.join(userMovies).map(lambda kv:(kv[1][0],kv[1][1]))
# print sortedLabels.take(1)
rankMetrics = RankingMetrics(sortedLabels)
print ("Mean Average Precision = %f" % rankMetrics.meanAveragePrecision)
print ("Mean Average Precision(at K=10) = %f" % rankMetrics.precisionAt(10))

Mean Average Precision = 0.072590
Mean Average Precision(at K=10) = 0.070732
